Annotated genome:

In [74]:
! wget ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz


URL transformed to HTTPS due to an HSTS policy
--2022-12-15 01:55:54--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17621408 (17M) [application/x-gzip]
Saving to: ‘GCA_001949185.1_Rvar_4.0_genomic.fna.gz.2’

GCA_001949185.1_Rva 100%[===================>]  16,80M  3,45MB/s    in 18s     

2022-12-15 01:56:12 (957 KB/s) - ‘GCA_001949185.1_Rvar_4.0_genomic.fna.gz.2’ saved [17621408/17621408]



Augustus reults with C.elegans as model: 

http://bioinf.uni-greifswald.de/webaugustus/prediction/show/0db58ea184c352e9018502c6df190081

Get protein sequences from AUGUSTUS output .gff

In [82]:
! ./getAnnoFasta.pl  augustus.gff

There are 17769 proteins

In [83]:
! grep ">" augustus.aa | wc -l

17769


tried to align peptides with Diamond, but result is totally incorrect (4 hits)

In [ ]:
! conda install -c bioconda diamond

In [68]:
!diamond makedb --in augustus.whole.aa  --db diamond_db

diamond v0.9.14.115 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU AGPL <https://www.gnu.org/licenses/agpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: augustus.whole.aa
Opening the database file...  [0.000699s]
Loading sequences...  [0.04987s]
Masking sequences...  [0.921379s]
Writing sequences...  [0.016453s]
Loading sequences...  [1e-05s]
Writing trailer...  [0.000216s]
Closing the input file...  [1.1e-05s]
Closing the database file...  [5.2e-05s]
Processed 16435 sequences, 7015168 letters.
Total time = 0.988807s


In [69]:
!diamond blastp -d diamond_db.dmnd -q peptides.fa -f 6 -o diamond_out.fa --sensitive


diamond v0.9.14.115 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU AGPL <https://www.gnu.org/licenses/agpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
#Target sequences to report alignments for: 25
Temporary directory: 
Opening the database...  [2.4e-05s]
Opening the input file...  [0.000768s]
Opening the output file...  [7.3e-05s]
Loading query sequences...  [3.4e-05s]
Masking queries...  [0.00242s]
Building query seed set...  [0.000213s]
Algorithm: Query-indexed
Building query histograms...  [0.000147s]
Allocating buffers...  [1.2e-05s]
Loading reference sequences...  [0.019119s]
Building reference histograms...  [0.010727s]
Allocating buffers...  [5e-05s]
Initializing temporary storage...  [0.002829s]
Processing query chunk 0, reference chunk 0, shape 0, index chunk 0.
Building reference index...  [0.021423s]
Building query index...  [0.001202s]
Building see

So here is run with Blastp

In [ ]:
! conda install -c bioconda blast

In [84]:
! makeblastdb -in augustus.aa -dbtype prot  -out blast_db



Building a new DB, current time: 12/15/2022 02:16:11
New DB name:   /home/pseudocephalus/BI/Workshop/project4/blast_db
New DB title:  augustus.aa
Sequence type: Protein
Deleted existing Protein BLAST database named /home/pseudocephalus/BI/Workshop/project4/blast_db
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 17769 sequences in 1.38341 seconds.


In [85]:
! blastp -db blast_db -query peptides.fa -outfmt "6 sseqid" -out blast_out   

Manually double checked with Ctrl+F, this is correct result: 34 DNA binding peptides are found in reference tardigrade

In [87]:
! sort -u blast_out > blast_sorted
! wc -l blast_sorted

34 blast_sorted


Simple script to parse results

In [89]:
import re
with open('augustus.aa') as ref, open('blast_sorted') as b, open('parsed_out', "w") as out:
    blast=b.read().split('\n')
    reference=ref.read().split('>')
    for item in reference:
        if item:
            name = re.match(r'.*?t1', item)
            if name.group() in blast:
                out.write(">"+item)

Loaded parsed_out to WolF PSORT, here is result. There are some genes marked as "nucleus", but what they are will be clear on BLASTing

НА СВОИХ ДАННЫХ ОТ АВГУСТУСА  - СМОТРЕТЬ НА НИХ https://wolfpsort.hgc.jp/results/aPR3a1cc04cfc4d7e1efaea2d4f3fc4c264.html

*** НА МИШИНЫХ ДАННЫХ - https://wolfpsort.hgc.jp/results/aKC18c5dca75a3f15f4e20c27853d9cf118.html

TargetP: only few proteins annotated as "Signal", seems like there is no link to paste


Our HMMER run available here:

www.ebi.ac.uk/Tools/hmmer/results/score/C7AE46BC-7D56-11ED-B53B-0C6F642C8881